## Building A Chatbot
Design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions


In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023445235410>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023445236D10>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish,\n\nIt's a pleasure to meet you!  As a Chief AI Engineer, you must be doing some fascinating work.\n\nWhat kind of AI projects are you currently involved in? I'm always eager to learn more about the advancements being made in the field. \n\n", response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 27, 'total_tokens': 89, 'completion_time': 0.128347179, 'prompt_time': 0.002622851, 'queue_time': None, 'total_time': 0.13097003000000002}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-206bf2b5-a020-4388-8086-957c31fafbb7-0', usage_metadata={'input_tokens': 27, 'output_tokens': 62, 'total_tokens': 89})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Krish and that you are a Chief AI Engineer.  \n\nIs there anything else you'd like me to know or help you with? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 102, 'total_tokens': 140, 'completion_time': 0.077622055, 'prompt_time': 0.006419323, 'queue_time': None, 'total_time': 0.084041378}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-59881325-1e32-4809-bc01-a2282753807a-0', usage_metadata={'input_tokens': 102, 'output_tokens': 38, 'total_tokens': 140})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
!pip install langchain_community


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [9]:
response.content

"Hi Krish! \n\nThat's a fantastic title!  It's great to meet a fellow AI enthusiast. What kind of exciting projects are you working on as a Chief AI Engineer? \n\nI'm always eager to learn about the innovative ways people are using AI.  \n\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Krish.  I remember that from our introduction! 😊  \n\nIs there anything else I can help you with today?  \n', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 102, 'total_tokens': 133, 'completion_time': 0.062638575, 'prompt_time': 0.005270483, 'queue_time': None, 'total_time': 0.06790905800000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c82f6373-80ef-487f-adcd-75da73778a9f-0', usage_metadata={'input_tokens': 102, 'output_tokens': 31, 'total_tokens': 133})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hi John! It's nice to meet you. \n\nWhat can I do for you today? 😊\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is John!  I remember that you told me.  \n\nIs there anything else I can help you with, John? 😊 \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish, it's nice to meet you!  \n\nHow can I help you today? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 36, 'total_tokens': 61, 'completion_time': 0.050690159, 'prompt_time': 0.002615331, 'queue_time': None, 'total_time': 0.05330549}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-af44b0ab-f443-4fbb-8b22-51cbcd4b9c97-0', usage_metadata={'input_tokens': 36, 'output_tokens': 25, 'total_tokens': 61})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hello Krish, it's nice to meet you!\n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 36, 'total_tokens': 61, 'completion_time': 0.050561489, 'prompt_time': 0.002677691, 'queue_time': None, 'total_time': 0.053239180000000004}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-39da6456-ceec-4afb-932e-45f257368d42-0', usage_metadata={'input_tokens': 36, 'output_tokens': 25, 'total_tokens': 61})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Krish.  I remember! 😊  \n\nIs there anything else I can help you with?\n'

In [19]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [20]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! 😊 \n\nआप कैसे हैं? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [21]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [22]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()


Error in callback coroutine: ValueError()


'नमस्ते कृष्ण, \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं! क्या आप मुझसे कुछ पूछना चाहते हैं? 😊\n'

In [23]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [24]:
response.content

'आपका नाम कृष्ण है।  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [25]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

d:\Python311\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

d:\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your taste preferences.  \n\nTo figure out what ice cream you like, you could think about:\n\n* **Your favorite flavors:** Do you prefer chocolate, fruity flavors, something nutty, or maybe something more unique?\n* **Your favorite toppings:**  Chocolate sauce, sprinkles, whipped cream, nuts? \n* **Past experiences:** What ice cream have you enjoyed before? \n\nLet me know if you want to brainstorm some ice cream flavors! 😊 🍦\n"

In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [28]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIs there anything else I can help you with? 😊\n"

In [30]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! Just let me know what it is. 😊  \n\n"